# New-York Times Api data collection

In [1]:
# import

import json
import requests
import csv
import pandas as pd
import time

## # 1st approach

In [2]:
# get the financial news article between 2019/01/01 and 2019/01/31

#response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20190101&end_date=20190131&q=financial&sort=relevance&page=2&api-key=7LYtg9bua3RqJzfQBTkKBZWdXsjGrerP")

In [3]:
# convert the response to a pureJSON string format

#response_json = response.json()
#response_json.keys()

In [4]:
# Extract articles from the response

#articles = response_json['response']['docs']
#articles

In [5]:
#def get_articles(file):
 #   """Return a list of dict. Each article is a dict."""
    
  #  articles_result = []
    
   # for i in range(len(file)):
    #    article_dict = {}
     #   article_dict['abstract'] = file[i]['abstract']
      #  article_dict['snippet'] = file[i]['snippet']
       # article_dict['lead_paragraph'] = file[i]['lead_paragraph']
        #article_dict['keywords'] = file[i]['keywords']
# article_dict['source'] = file[i]['source']
        #article_dict['date'] = file[i]['pub_date']
        #article_dict['url'] = file[i]['web_url']
        
        #articles_result.append(article_dict)
    #return articles_result

In [6]:
# Convert the list of dict in a dataframe and print the output

#df_articles = pd.DataFrame(get_articles(articles))
#df_articles

The problem is that we only get the news article 10 by 10 as there are 10 articles by page.

In [7]:
#df_articles.keywords[2]

## # 2nd approach (smarter)

In [8]:
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

In [9]:
api_key = '7LYtg9bua3RqJzfQBTkKBZWdXsjGrerP'

In [10]:
def get_response(url:str, api_key:str, query:str, begin_date:str, end_date:str, nb_pages:int)->list:
    """For a given url, return all the news articles in a list composed of dictionnaries.
    Each element corresponds to the results for each page (10 pages = 10 elements) 
    :url the url of the page to scrap
    """
    
    response_list = []
    #pages = 10
    
    for i in range(nb_pages):
        response_json = {}
        parameters = {
            #'q' : 'Financial',
            #'begin_date' : '20190101',
            #'end_date' : '20190131',
            'q' : query,
            'api-key' : api_key,
            'begin_date' : begin_date,
            'end_date' : end_date,
            'sort' : 'relevance',
            'page' : i
        }
        response = requests.get(url, params=parameters)
        response_json = response.json()
        response_list.append(response_json)
        time.sleep(6)
    
    return response_list

In [11]:
#response = get_response(url)
response = get_response(url, api_key, 'Financial', '20100101', '20200101', 10)
len(response)      # is equal to the number of pages retrieved

10

In [12]:
def get_articles(file):
    """Return a list of dict. Each article is a dict."""
    
    articles_result = []
    
    for i in range(len(file)):
        article_dict = {}
        article_dict['abstract'] = file[i]['abstract']
        article_dict['snippet'] = file[i]['snippet']
        article_dict['lead_paragraph'] = file[i]['lead_paragraph']
        article_dict['keywords'] = file[i]['keywords']
       #article_dict['source'] = file[i]['source']
        article_dict['date'] = file[i]['pub_date']
        article_dict['url'] = file[i]['web_url']
        
        articles_result.append(article_dict)
    return articles_result

In [13]:
def get_dataframe(response:dict):
    """Return a dataframe. Take the response to the request in json format as parameters. """
    
    df_articles = pd.DataFrame(get_articles(response[0]['response']['docs']))
    nb_articles = 9
    
    for i in range(1,len(response)):
        for j in range(10):
            nb_articles = nb_articles +1
            df_articles.loc[nb_articles] = get_articles(response[i]['response']['docs'])[j]
            
    return df_articles

In [14]:
# transpose the response into a clean dataframe

df_articles = get_dataframe(response)

In [15]:
df_articles

,abstract,snippet,lead_paragraph,keywords,date,url
0,Wall Street wants you to stay ignorant. @MrsDo...,Wall Street wants you to stay ignorant. @MrsDo...,"Haley Sacks, 28, doesn’t just want women to sa...","[{'name': 'subject', 'value': 'Banking and Fin...",2019-12-24T16:58:34+0000,https://www.nytimes.com/2019/12/24/style/finan...
1,Congress could do much more to protect America...,Congress could do much more to protect America...,Congress invited predatory for-profit colleges...,"[{'name': 'subject', 'value': 'Veterans', 'ran...",2020-01-01T00:18:54+0000,https://www.nytimes.com/2019/12/31/opinion/for...
2,An agreement between college counselors and th...,An agreement between college counselors and th...,WASHINGTON — The financial aid stranglehold on...,"[{'name': 'subject', 'value': 'ADMISSIONS STAN...",2019-12-21T17:42:19+0000,https://www.nytimes.com/2019/12/21/us/politics...
3,The fateful decision in 1979 to admit Mohammed...,The fateful decision in 1979 to admit Mohammed...,"One late fall evening 40 years ago, a worn-out...","[{'name': 'glocations', 'value': 'Iran', 'rank...",2019-12-29T05:01:11+0000,https://www.nytimes.com/2019/12/29/world/middl...
4,"Andrew Bailey, the head of a financial regulat...","Andrew Bailey, the head of a financial regulat...",The British government on Friday named Andrew ...,"[{'name': 'persons', 'value': 'Bailey, Andrew ...",2019-12-20T08:53:38+0000,https://www.nytimes.com/2019/12/20/business/an...
...,...,...,...,...,...,...
95,"Banks including Capital One, U.S. Bancorp, Bar...","Banks including Capital One, U.S. Bancorp, Bar...",Federal Reserve Board members voted on Thursda...,"[{'name': 'subject', 'value': 'Banking and Fin...",2019-10-10T22:39:56+0000,https://www.nytimes.com/2019/10/10/business/ec...
96,Technology executives like WeWork’s Adam Neuma...,Technology executives like WeWork’s Adam Neuma...,"The banking industry, which has consistently b...","[{'name': 'subject', 'value': 'Banking and Fin...",2019-10-10T15:48:21+0000,https://www.nytimes.com/2019/10/10/opinion/wew...
97,"Mergers, acquisitions, shorter degree programs...","Mergers, acquisitions, shorter degree programs...","When Steve Thorsett crunched the numbers, thin...","[{'name': 'subject', 'value': 'Colleges and Un...",2019-10-10T09:00:20+0000,https://www.nytimes.com/2019/10/10/education/l...
98,The justices could take up a case on whether t...,The justices could take up a case on whether t...,The Supreme Court’s 2019-20 term started on Mo...,"[{'name': 'subject', 'value': 'Law and Legisla...",2019-10-08T10:00:08+0000,https://www.nytimes.com/2019/10/08/opinion/tru...


In [16]:
def get_keywords(df):
    return df['keywords']

In [17]:
get_keywords(df_articles)[1]

[{'name': 'subject', 'value': 'Veterans', 'rank': 1, 'major': 'N'},
 {'name': 'subject', 'value': 'For-Profit Schools', 'rank': 2, 'major': 'N'},
 {'name': 'subject',
  'value': 'Financial Aid (Education)',
  'rank': 3,
  'major': 'N'},
 {'name': 'subject', 'value': 'Frauds and Swindling', 'rank': 4, 'major': 'N'},
 {'name': 'subject',
  'value': 'Colleges and Universities',
  'rank': 5,
  'major': 'N'},
 {'name': 'organizations',
  'value': 'Veterans Affairs Department',
  'rank': 6,
  'major': 'N'},
 {'name': 'organizations',
  'value': 'Federal Trade Commission',
  'rank': 7,
  'major': 'N'},
 {'name': 'organizations',
  'value': 'University of Phoenix',
  'rank': 8,
  'major': 'N'},
 {'name': 'organizations',
  'value': 'Career Education Corporation',
  'rank': 9,
  'major': 'N'}]